In [ ]:
import gensim.downloader as api
import numpy as np
import pandas as pd
import random

In [ ]:
# Load the dataset
synonym_test_dataset = pd.read_csv('A2-DataSet/synonym.csv')

print(synonym_test_dataset.head())
print(synonym_test_dataset.info())

# Task 1

In [ ]:
# takes up to about 5 minuntes to load
w2v_model = api.load('word2vec-google-news-300')

In [ ]:
def assign_label(question_word, correct_answer, closest_synonym, guess_words, model):
    # Check if question word and at least one guess word are in the vocabulary
    if question_word in model.key_to_index and any(word in model.key_to_index for word in guess_words):
        # Check if the closest guess is correct
        if closest_synonym is not None and closest_synonym == correct_answer:
            return "correct"
        else:
            return "wrong"
    else:
        return "guess"

def closest_synonym(query, list_of_guess_words, model):
    closest_synonym = random.choice(list_of_guess_words) if list_of_guess_words else None
    max_similarity = -1

    # Check if the query word is in the model's vocabulary
    if query not in model.key_to_index:
        print(f"'{query}' is not in the vocabulary.")
        # Return a random guess word if the query is not in the vocabulary
        return closest_synonym

    for guess_word in list_of_guess_words:
        # Check if the guess word is in the model's vocabulary
        if guess_word in model.key_to_index:
            try:
                sim_score = model.similarity(query, guess_word)
                if sim_score > max_similarity:
                    closest_synonym = guess_word
                    max_similarity = sim_score
            except KeyError:
                # Handle the error if the word is not in the model's vocabulary
                continue
        else:
            print(f"'{guess_word}' is not in the vocabulary.")
            # Return a random guess word if the query is not in the vocabulary
            return closest_synonym

    return closest_synonym


# Function to proSV file and apply the closest_synonym function
def process_csv(file_path, model_name, model):
    
    # Size of the Vocabulary
    vocab_size = len(model.key_to_index)

    question_words = []
    answer_words = []
    guess_words = []
    labels = []
    C = 0
    V = 0
    
    # Read the CSV file into a Pandas DataFrame, skipping the first row
    synonym_test_dataset = pd.read_csv(file_path)

    # Process each row in the DataFrame
    for index, row in synonym_test_dataset.iterrows():
        # Split the row into words based on comma
        words = row.to_list()
        #print(words)

        # Store the first word in 'query' and the rest in 'list_of_guess_words'
        query = words[0]
        #print(query)
        answer = words[1]
        #print(answer)
        list_of_guess_words = words[2:]
        #print(list_of_guess_words)

        # Call the 'closest_synonym' function and store the result
        result = closest_synonym(query, list_of_guess_words, model)
        
        question_words.append(query)
        answer_words.append(answer)
        guess_words.append(result)
        label = assign_label(query, answer, result, list_of_guess_words, model)
        labels.append(label)
        
        if label == 'correct':
            C += 1
        if label != 'guess':
            V += 1
    
    if V == 0:
        accuracy = 0
    else: 
        accuracy = C/V
    
    results_df = pd.DataFrame({'question_word': question_words, 'answer_word': answer_words, 'guess_word': guess_words, 'label': labels})    
    results_df.to_csv(f"{model_name}-details.csv", index=False)
    
    analysis_df = pd.DataFrame({'model_name': [model_name], 'vocab_size': [vocab_size], 'C': [C], 'V': [V], 'accuracy': accuracy})    
    analysis_df.to_csv('analysis.csv', mode='a', index=False, header=not pd.io.common.file_exists('analysis.csv'))
    
    print(results_df)
    
file_path = 'A2-DataSet/synonym.csv' 
processed_results = process_csv(file_path, 'word2vec-google-news-300', w2v_model)

# Task 2 

Note: we got the corpus names from the offical gensim GitHub page (https://github.com/piskvorky/gensim-data).

In [ ]:
# takes about 3 minuntes to load
gigaword_model = api.load('glove-wiki-gigaword-300')

In [ ]:
# takes about 8 minuntes to load
fasttext_model = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
# takes about 5 minuntes to load
glove_twitter_200_model = api.load('glove-twitter-200')

In [ ]:
# takes about 2 minuntes to load
glove_twitter_25_model = api.load('glove-twitter-25')

In [ ]:
file_path = 'A2-DataSet/synonym.csv' 

processed_results = process_csv(file_path, 'glove-wiki-gigaword-300', gigaword_model)
processed_results = process_csv(file_path, 'fasttext-wiki-news-subwords-300', fasttext_model)
processed_results = process_csv(file_path, 'glove-twitter-200', glove_twitter_200_model)
processed_results = process_csv(file_path, 'glove-twitter-25', glove_twitter_25_model)

# Task 3

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def read_and_process_docs(file_paths):
    
    documents = []
    
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            file_contents = file.read()
            documents.append(process_text(file_contents))
    
    print(documents)
    print(f"Processed {len(documents)} documents.")
    return documents


def process_text(text):
        # tokenize the text into sentences
        sentences = sent_tokenize(text)
        sentences_list = []
        
        for sentence in sentences:
            # tokenize the text
            tokens = word_tokenize(sentence)
            
            # remove punctuation and numbers and convert to lowercase
            tokens = [word.lower() for word in tokens if word.isalpha()]
            
            # join the tokens into a single string with each word separated by a space
            tokens_string = ' '.join(tokens)
            
            # return the processed sentence
            sentences_list.append(tokens_string)
            
        
        return sentences_list
    
file_paths = ['Online-Books/Moby_Dick_Or_The_Whale.txt', 'Online-Books/Peter_Pan.txt', 'Online-Books/Pride_and_Prejudice.txt', 'Online-Books/The_Complete_Works_of_William_Shakespeare.txt', 'Online-Books/The_Importance_of_Being_Earnest_A_Trivial_Comedy_for_Serious_People.txt', 'Online-Books/Winnie_the_Pooh.txt']
read_and_process_docs(file_paths)
    